In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import pandas as pd

conc_dataset_path = '/content/drive/My Drive/CS 5787/Project/OpenCHAIR-Adjectives/Concreteness_ratings_Brysbaert_et_al_BRM.xlsx'
word_conc = pd.read_excel(conc_dataset_path)[['Word','Conc.M']].set_index("Word").to_dict()['Conc.M']

In [3]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [4]:
from datasets import load_dataset
print("Loading Dataset\n")
och_dataset = load_dataset("moranyanuka/OpenCHAIR")['test']

Loading Dataset



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.10k [00:00<?, ?B/s]

test-00000-of-00002.parquet:   0%|          | 0.00/375M [00:00<?, ?B/s]

test-00001-of-00002.parquet:   0%|          | 0.00/376M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4863 [00:00<?, ? examples/s]

In [6]:
generations_file_path = '/content/drive/My Drive/CS 5787/Project/OpenCHAIR-Adjectives/example_gen_file.csv'
df = pd.read_csv(generations_file_path)

In [7]:
df['gt_caption'] = och_dataset['text'][:len(df)]

In [30]:
import spacy
from tqdm.auto import tqdm
spacy.require_gpu()

def is_concrete(noun, concretness, t=2.5):
    if noun in concretness:
        return concretness[noun] > t
    return False

def extract_objs(captions, conc_df):
    nlp = spacy.load("en_core_web_sm")
    objs = []
    for caption in tqdm(captions):
        doc = nlp(caption.lower())
        cur_objs = [token.lemma_ for token in doc if token.pos_ == 'ADJ' and is_concrete(token.lemma_, conc_df)]
        objs.append(cur_objs)
    return objs

In [31]:
df['generated_adjs'] = extract_objs(df.generated_caption.tolist(), word_conc)

  0%|          | 0/2000 [00:00<?, ?it/s]

In [32]:
print(df['generated_adjs'][:40])

0                        []
1                        []
2                        []
3                     [old]
4                        []
5                     [red]
6                        []
7                        []
8                        []
9                        []
10                       []
11                       []
12                       []
13                       []
14                       []
15                       []
16            [green, pink]
17                       []
18                       []
19                       []
20                  [green]
21                       []
22                [elderly]
23                       []
24                       []
25                       []
26                       []
27                       []
28                       []
29    [redhead, red, green]
30                       []
31                  [green]
32                       []
33                 [wooden]
34                       []
35                  

In [33]:
print(df)

                                      generated_caption  \
0                                    a wolf and a moose   
1                                           an air show   
2                 a boy riding a bike in a neighborhood   
3                                              old keys   
4                        a group of people on the beach   
...                                                 ...   
1995                                 a glass of whiskey   
1996    ga ga ga ga ga ga ga ga ga ga ga ga ga ga ga ga   
1997                                    a gothic church   
1998  a father and his daughter riding a tri tri tri...   
1999                              a stack of steel bars   

                                             gt_caption generated_adjs  
0     A fighter jet in a combat scene with a missile...             []  
1     A young boy is riding his bike through a neigh...             []  
2     A group of people at the beach, with some of t...             []  